# Sprawozdanie - funkcje skrótu
## Bartosz Skrzypczak 155832

Załadowanie potrzebnych bibliotek

In [2]:
import hashlib
import timeit
import random
import string
import matplotlib.pyplot as plt
import pandas as pd

Funkcja generate_hashes przyjmuje tekst wejściowy, koduje go do formatu bajtowego, a następnie generuje i zwraca skróty (w formie szesnastkowej) dla różnych funkcji skrótu, takich jak MD5, SHA-1, SHA-224, SHA-256, SHA-384, SHA-512 oraz SHA3 (w wariantach 224, 256, 384 i 512 bitów).

In [3]:
def generate_hashes(input_text):
    input_bytes = input_text.encode('utf-8')
    return {
        'MD5': hashlib.md5(input_bytes).hexdigest(),
        'SHA-1': hashlib.sha1(input_bytes).hexdigest(),
        'SHA-224': hashlib.sha224(input_bytes).hexdigest(),
        'SHA-256': hashlib.sha256(input_bytes).hexdigest(),
        'SHA-384': hashlib.sha384(input_bytes).hexdigest(),
        'SHA-512': hashlib.sha512(input_bytes).hexdigest(),
        'SHA3-224': hashlib.sha3_224(input_bytes).hexdigest(),
        'SHA3-256': hashlib.sha3_256(input_bytes).hexdigest(),
        'SHA3-384': hashlib.sha3_384(input_bytes).hexdigest(),
        'SHA3-512': hashlib.sha3_512(input_bytes).hexdigest()
    }

Funkcja benchmark_hash_function mierzy czas wykonania wybranej funkcji skrótu, obliczając jej wynik dla podanego tekstu wejściowego, a następnie zwraca czas wykonania tej operacji, powtarzanej 1000 razy. W kodzie głównym generowany jest zestaw danych wejściowych o różnych długościach (10 000, 50 000, 100 000, 500 000, 1 000 000 znaków), a następnie dla każdej długości tekstu mierzone są czasy obliczeń dla różnych funkcji skrótu (MD5, SHA-1, SHA-224, SHA-256, SHA-384, SHA-512 oraz SHA3). Na końcu wyniki są zapisywane w tabeli (DataFrame), zawierającej czasy wykonania oraz długości wygenerowanych skrótów.

In [6]:
def benchmark_hash_function(hash_func, input_text):
    def wrapper():
        hash_func(input_text.encode('utf-8')).hexdigest()
    return timeit.timeit(wrapper, number=1000)

input_lengths = [10_000, 50_000, 100_000, 500_000, 1_000_000]
results = []

for length in input_lengths:
    input_text = ''.join(random.choices(string.ascii_letters + string.digits, k=length))
    row = {'Input Length': length}
    for name, func in {
        'MD5': hashlib.md5, 'SHA-1': hashlib.sha1,
        'SHA-224': hashlib.sha224, 'SHA-256': hashlib.sha256,
        'SHA-384': hashlib.sha384, 'SHA-512': hashlib.sha512,
        'SHA3-224': hashlib.sha3_224, 'SHA3-256': hashlib.sha3_256,
        'SHA3-384': hashlib.sha3_384, 'SHA3-512': hashlib.sha3_512
    }.items():
        exec_time = benchmark_hash_function(func, input_text)
        hash_len = len(func(input_text.encode('utf-8')).hexdigest())
        row[f'{name} Time'] = exec_time
        row[f'{name} Length'] = hash_len
    results.append(row)

df = pd.DataFrame(results)
df


,Input Length,MD5 Time,MD5 Length,SHA-1 Time,SHA-1 Length,SHA-224 Time,SHA-224 Length,SHA-256 Time,SHA-256 Length,SHA-384 Time,...,SHA-512 Time,SHA-512 Length,SHA3-224 Time,SHA3-224 Length,SHA3-256 Time,SHA3-256 Length,SHA3-384 Time,SHA3-384 Length,SHA3-512 Time,SHA3-512 Length
0,10000,0.025140,32,0.005931,40,0.005582,56,0.005344,64,0.008262,...,0.007873,128,0.014933,56,0.015096,64,0.019755,96,0.027307,128
1,50000,0.072501,32,0.020667,40,0.020283,56,0.020199,64,0.033627,...,0.033583,128,0.069308,56,0.073823,64,0.096902,96,0.140486,128
2,100000,0.143639,32,0.041085,40,0.040588,56,0.040555,64,0.068325,...,0.067501,128,0.138369,56,0.146205,64,0.188733,96,0.268959,128
3,500000,0.703448,32,0.197757,40,0.195943,56,0.196156,64,0.368713,...,0.337529,128,0.696562,56,0.730438,64,0.946040,96,1.404087,128
4,1000000,1.595153,32,0.455484,40,0.443682,56,0.443482,64,0.736046,...,0.749075,128,1.544780,56,1.620876,64,2.057936,96,2.897416,128


### Zadanie 2

### Szybkość działania funkcji skrótu

Dla każdego rozmiaru danych wejściowych (od 10 000 do 1 000 000 znaków), czas obliczenia skrótu wzrasta liniowo. Oto ogólne obserwacje:

 - Najszybsze funkcje:

    - SHA-1 i SHA-224 – są najwydajniejsze przy wszystkich rozmiarach wejściowych.

    - SHA-256 także wypada bardzo dobrze i stabilnie.

 - Wolniejsze funkcje:

    - SHA-3 (wszystkie warianty) są znacznie wolniejsze, zwłaszcza SHA3-512, która dla 1 000 000 znaków potrzebuje ~2,9 sekundy, czyli najwięcej ze wszystkich.

    - MD5 osiąga porównywalne wyniki do najszybszych funkcji SHA-3 czyli SHA3-224 i SHA3-256, jednak jest prawie 4x wolniejszy od SHA-1, SHA-224 i SHA-256


### Długość skrótu wyjściowego

Skróty o najmniejszej długości generuje algorytm MD5, a ich długość wynosi 32 znaki. Najdłuższe skróty mają a 128 znaków - generowane przez SHA-512 i SHA3-512.

Długość skrótów z grupy SHA2 i SHA3 jest taka sama, dla wariantów o tym samym "kodzie" - tzn. np. długość skrótu SHA-512 jest taka sama jak SHA3-512. To oznaczenie liczbowe, odpowiadia liczbie bitów, na jakies zapisany jest hash. Korzystamy z systemu szesnastkowego, zatem przykładowe 512 należałoby podzielić przez 4 - wtedy uzyskamy rzeczywistą długość ciągu tj. 128 znaków.

In [8]:
short_input = "hej"
md5_hash = hashlib.md5(short_input.encode()).hexdigest()
print(f"MD5 hash of '{short_input}': {md5_hash}")

MD5 hash of 'hej': 541c57960bb997942655d14e3b9607f9


### Zadanie 3
Sprawdzić czy wartość funkcji jest powszechnie znany można np. na tej stronie: https://crackstation.net/

Po sprawdzeniu okazuje się, że jest znana zatem a jej odszyfrowanie zajmuje ułamek sekundy. 
Oznacza to, że użycie funkcji MD5 dla tak krótkich tekstów nie gwarantuje żadnego bezpieczeństwa, bo wiedząc, że hash jest wynikiem funkcji skrótu
jego odszyfrowanie nie stanowi żadnej trudności.

### Zadanie 4

Nie, nie jest bezpieczna. MD5 nie spełnia współczesnych standardów kryptograficznych i nie powinna być używana do zabezpieczania danych, w szczególności haseł czy podpisów cyfrowych.

Dlaczego MD5 jest niebezpieczna?
- Kolizje zostały oficjalnie znalezione – czyli możliwe jest wygenerowanie dwóch różnych danych wejściowych, które dają identyczny hash.

    - W 2004 roku Xiaoyun Wang i Hongbo Yu opublikowali pierwszy praktyczny atak kolizyjny.

    - W 2008 udało się podrobić certyfikat SSL z użyciem kolizji MD5.

    - Google i Centrum Badawcze CWI w 2017 roku przeprowadziły atak SHAttered – generując dwie różne PDF-y z tym samym hash MD5.

- Brak odporności na preimage – istnieją sposoby na odnalezienie danych wejściowych pasujących do danego skrótu.

- MD5 jest bardzo szybki, co sprzyja atakom brute-force i wykorzystaniu rainbow tables, szczególnie w przypadku krótkich haseł.

### Zadanie 5

Funkcja first_n_bits(h, n=12) przekształca skrót w postaci szesnastkowej na reprezentację binarną i zwraca tylko pierwsze n bitów, gdzie domyślnie n wynosi 12.

Funkcja count_collisions(hash_function, num_trials=10000, bit_length=12) przeprowadza test kolizji dla zadanej funkcji skrótu. Dla każdego z num_trials losowo generowanego ciągu wejściowego (o długości 10 znaków), obliczany jest skrót, a następnie wyciągane są pierwsze n bitów z tego skrótu. Jeżeli te bity zostały już wcześniej napotkane, zwiększa się liczba kolizji. Na końcu funkcja zwraca całkowitą liczbę kolizji.

Testowanie jest przeprowadzane dla dwóch funkcji skrótu: SHA-256 i SHA3-256, z ustawioną liczbą prób na 10,000 i liczbą badanych bitów na 12.

In [28]:
def first_n_bits(h, n=12):
    return bin(int(h, 16))[2:].zfill(256)[:n]

# Funkcja do zliczania kolizji
def count_collisions(hash_function, num_trials=10000, bit_length=12):
    seen = set()
    collisions = 0
    for _ in range(num_trials):
        random_input = ''.join(random.choices(string.ascii_letters + string.digits, k=10))
        digest = hash_function(random_input.encode()).hexdigest()
        bits = first_n_bits(digest, bit_length)
        if bits in seen:
            collisions += 1
        else:
            seen.add(bits)
    return collisions


# Testowanie dla SHA-256
collisions_sha256 = count_collisions(hashlib.sha256, num_trials=10000, bit_length=12)
print(f"Kolizje SHA-256 (12 bitów, 10000 prób): {collisions_sha256}")

# Testowanie dla SHA-3-256
collisions_sha3_256 = count_collisions(hashlib.sha3_256, num_trials=10000, bit_length=12)
print(f"Kolizje SHA3-256 (12 bitów, 10000 prób): {collisions_sha3_256}")

Kolizje SHA-256 (12 bitów, 10000 prób): 6239
Kolizje SHA3-256 (12 bitów, 10000 prób): 6293


### Zadanie 6

Funkcja hamming_distance oblicza liczbę różniących się bitów między dwoma skrótami w formie szesnastkowej, używając operacji XOR. Funkcja test_sac testuje kryterium SAC (Strict Avalanche Criterion) dla wybranej funkcji skrótu, mierząc, jak bardzo zmieniają się bity wyniku po zmianie jednego bitu w wejściu. Proces ten powtarza się przez określoną liczbę prób, a na koniec obliczana jest średnia liczba zmienionych bitów. Testowanie zostało przeprowadzone dla SHA-1, a wynik pokazuje, jak dobrze funkcja spełnia kryterium SAC.

In [70]:
def hamming_distance(h1, h2):
    return bin(int(h1, 16) ^ int(h2, 16)).count('1')

# Funkcja do badania SAC
def test_sac(hash_function, num_trials=100, input_length=16):
    bits_changed = []

    for _ in range(num_trials):
        original_input = ''.join(random.choices('01', k=input_length))
        original_hash = hash_function(original_input.encode()).hexdigest()

        mutated_input = list(original_input)
        mutated_input[random.randint(0, len(mutated_input) - 1)] = '1' if mutated_input[random.randint(0, len(mutated_input) - 1)] == '0' else '0'
        mutated_input = ''.join(mutated_input)
        mutated_hash = hash_function(mutated_input.encode()).hexdigest()

        diff_bits = hamming_distance(original_hash, mutated_hash)
        bits_changed.append(diff_bits)

    average_changed_bits = sum(bits_changed) / len(bits_changed)
    return average_changed_bits

# Testowanie SAC dla SHA-1
average_sha1 = test_sac(hashlib.sha1, num_trials=100)
print(f"Średnia liczba zmienionych bitów w SHA-1: {average_sha1}")

Średnia liczba zmienionych bitów w SHA-1: 49.19


### Rola soli

Rola soli w tworzeniu skrótów polega na zwiększeniu bezpieczeństwa przez dodanie losowego ciągu danych (tzw. soli) do wejściowego hasła lub wiadomości przed obliczeniem jego skrótu. Solenie hasła utrudnia ataki takie jak ataki słownikowe czy ataki typu rainbow tables, ponieważ każde hasło, nawet jeśli jest takie samo, będzie miało inny skrót ze względu na unikalną sól. Dzięki temu, atakujący nie może wykorzystać gotowych tabel skrótów (np. dla popularnych haseł), a także zyskuje się odporność na ataki brute force, ponieważ każde hasło jest „rozmyte” przez sól. Sól jest przechowywana w bazie danych razem z hasłem, ale nie powinna być używana w procesie tworzenia skrótu bez dodatkowego losowego elementu.